In [1]:
from carbon.helpers.cryptocompare import CryptoCompare
import pandas as pd
from matplotlib import pyplot as plt
import os
print( "{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CryptoCompare))

CryptoCompare v1.4 (01/Feb/2023)


# Crypto Compare

https://min-api.cryptocompare.com/documentation

## Selection

Please enter the pairs for the `SELECTION` data collection below

In [2]:
selection = "MIM/LTC, BNT/LINK, NMR/USD"

In [13]:
pd.read_pickle("cryptocompare/SELECTION.pickle").head()

,BNT/LINK,MIM/LTC,NMR/USD
datetime,,,
2017-08-11,0.0,0.0,34.72
2017-08-12,0.0,0.0,43.90
2017-08-13,0.0,0.0,36.76
2017-08-14,0.0,0.0,34.62
2017-08-15,0.0,0.0,31.30


In [14]:
# !cp cryptocompare/SELECTION.pickle .
# !cp cryptocompare/SELECTION.pickle ../../../JupyLiteCarbonSim/content/data
# !cp cryptocompare/SELECTION.pickle ../../../BinderCarbonSimulator/Frozen/data

## SetUp

If you have a private API key, run `export CCAPIKEY=<YOURKEY>` before you launch Jupyter on the same shell. In case no key is found the free API is used. You can also provide the API key in the constructor, but this is not recommended for security reasons.

In [5]:
try:
    CC = CryptoCompare(apikeyname="CCAPIKEY")
except:
    CC = CryptoCompare(apikey=True)

print("[CryptoCompare] key digest", CC.keydigest[:4])

Can't find API key CCAPIKEY in environment variables.
Use `export CCAPIKEY=<value>` to set it BEFORE you launch Jupyter
[CryptoCompare] key digest 0000


## Coin lists

Here we populate the various coin lists that we will use to download data. They are as follows:

- `coins_for_ccy`, `ccies`: creates a table for each of the currencies 
- `coins_for_cross`: creates a single cross table
- `usdstables`: creates a single table against USD

We also define the list of items **not** to produce, `excludes`. Finally there is an `includes` list that, if present, specifies the _only_ items to be produced.

In [6]:
coins_for_ccy ="ETH, BTC, BNB, XRP, ADA, DOGE, MATIC, SOL, OKB, DOT, LTC, TRX, AVAX, ATOM"
coins_for_cross ="ETH, BTC, BNB, XRP, ADA, DOGE, MATIC, SOL, OKB, DOT, LTC, TRX, AVAX, ATOM"
usdstables = "USDT, USDC, UST, BUSD"
ccies = "USD, BTC, ETH"

#excludes = ['COINS-CROSS', 'STABLES-USD', 'COINS-USD', 'COINS-BTC', 'COINS-ETH']
includes = ["SELECTION"]

The table `dltable0` has as keys the filename, and the data is a tuple of pairs. The table `dltable` contains the final downloads, the difference being the `excludes` to avoid redownloading data that is not needed. 

**YOU MUST RESTART THE KERNEL IF YOU MAKE CHANGES TO INCLUDES OR EXCLUDES**.

In [7]:
dltable0 = {
    "COINS-CROSS": CC.create_pairs(coins_for_cross),
    "STABLES-USD": CC.create_pairs(usdstables, "USD"),
    **{
        f"COINS-{ccy}": CC.create_pairs(coins_for_ccy, ccy)
        for ccy in CC.coinlist(ccies)
    },
    "SELECTION": CC.coinlist(selection, aspt=True),
}
try:
    dltable = {k:v for k, v in dltable0.items() if k in includes}
    print("restricted execution to the pairs in `includes`", includes)
except:
    try:
        dltable = {k:v for k, v in dltable0.items() if not k in excludes}
        print("the items in `excludes` were excluded", excludes)
    except:
        dltable = dltable0
        print("no `includes` or `excludes` info present; all tables are being process")
dltable0.keys(), dltable.keys()

restricted execution to the pairs in `includes` ['SELECTION']


(dict_keys(['COINS-CROSS', 'STABLES-USD', 'COINS-USD', 'COINS-BTC', 'COINS-ETH', 'SELECTION']),
 dict_keys(['SELECTION']))

## Data download

### Raw tables

In [8]:
for item, pairs in dltable.items():
    print("Downloading raw table", item, len(pairs))
    results = {
        (fsym, tsym): CC.query_dailypair(fsym=fsym, tsym=tsym)
        for fsym, tsym in pairs
    }
    df = pd.concat(results, axis=1)
    df.to_pickle(CC.datafn(f"{item}-RAW.pickle"))

[query] https://min-api.cryptocompare.com/data/v2/histoday {'fsym': 'MIM', 'tsym': 'LTC', 'limit': 2000} [0000]
[query] https://min-api.cryptocompare.com/data/v2/histoday {'fsym': 'BNT', 'tsym': 'LINK', 'limit': 2000} [0000]
[query] https://min-api.cryptocompare.com/data/v2/histoday {'fsym': 'NMR', 'tsym': 'USD', 'limit': 2000} [0000]


In [9]:
!ls cryptocompare

COINS-BTC-RAW.pickle      COINS-USD-RAW.pickle      isoccy.csv
COINS-BTC.pickle          COINS-USD.pickle          query_allexchanges.pickle
COINS-CROSS-RAW.pickle    SELECTION-RAW.pickle      query_coinlist.pickle
COINS-CROSS.pickle        SELECTION.pickle          query_indexlist.pickle
COINS-ETH-RAW.pickle      STABLES-USD-RAW.pickle
COINS-ETH.pickle          STABLES-USD.pickle


### Reformatted tables

In [10]:
for item in dltable:
    print("Converting raw table", item)
    df = pd.read_pickle(CC.datafn(f"{item}-RAW.pickle"))
    df = CC.reformat_raw_df(df)
    df.to_pickle(CC.datafn(f"{item}.pickle"))

Converting raw table SELECTION


## Review tables

In [11]:
!ls cryptocompare

COINS-BTC-RAW.pickle      COINS-USD-RAW.pickle      isoccy.csv
COINS-BTC.pickle          COINS-USD.pickle          query_allexchanges.pickle
COINS-CROSS-RAW.pickle    SELECTION-RAW.pickle      query_coinlist.pickle
COINS-CROSS.pickle        SELECTION.pickle          query_indexlist.pickle
COINS-ETH-RAW.pickle      STABLES-USD-RAW.pickle
COINS-ETH.pickle          STABLES-USD.pickle


In [12]:
df = pd.read_pickle("cryptocompare/SELECTION.pickle")
df

,BNT/LINK,MIM/LTC,NMR/USD
datetime,,,
2017-08-11,0.00000,0.00000,34.72
2017-08-12,0.00000,0.00000,43.90
2017-08-13,0.00000,0.00000,36.76
2017-08-14,0.00000,0.00000,34.62
2017-08-15,0.00000,0.00000,31.30
...,...,...,...
2023-01-28,0.05941,0.01104,16.60
2023-01-29,0.05952,0.01044,17.07
2023-01-30,0.05965,0.01084,16.06
